This notebook is meant as a guide to testing out wifi localization on the MAC addresses that are visible from your local wifi interface.

You should start by cloning this repository, and running this notebook from inside the cloned directory.

In [1]:
import os
import re

The following will determine that you have a sqlite database.

In [2]:
if "us.sqlite" in os.listdir("data"):
    print "You appear to have the us.sqlite database in the correct location."
else:
    print """You don't appear to have the us.sqlite database.  
Please download https://cdn.radiocells.org/us.sqlite ,
and put the database in the ./data/ directory.
"""

You appear to have the us.sqlite database in the correct location.


The following cell will determine if you have sqlite3.

In [3]:
if "sqlite3" in os.listdir("/usr/bin/"):
    print "You have sqlite3."
else:
    print """You don't appear to have sqlite3.  You can download it here: https://www.sqlite.org/download.html
If you think you already have it in a nonstandard location, you can try running the commands below anyway."""

You have sqlite3.


The following will build the localization binary. This will probably print nothing if everything goes well.

In [10]:
build_command = "g++ localization_main.cc lib/estimation_functions.cc lib/query_functions.cc lib/drop_old.cc -o localization -std=c++11 -l sqlite3"
with os.popen(build_command) as build_output:
    print build_output.read()

The following will get get all nearby bssids and corresponding rssis.

On Mac OS:

In [5]:
def process_raw_lines_from_apple_airport(raw_lines):
    items = list()
    pat = r"([a-z0-9]{2}:){5}[a-z0-9]{2} -[0-9]+"
    for line in raw_lines:
        m = re.search(pat, line)
        if m:
            items.append(m.group(0).upper().replace(":", ""))
    return items

def get_bssids_and_rssis_via_apple_airport():
    airport_command = "/System/Library/PrivateFrameworks/Apple80211.framework/Versions/Current/Resources/airport -s"
    with os.popen(airport_command) as f:
        raw_lines = f.readlines()
    results = process_raw_lines_from_apple_airport(raw_lines)
    return " ".join(results)

bssids_and_rssis = get_bssids_and_rssis_via_apple_airport()

print bssids_and_rssis

4C3275C4C4CC -73 2C3033C23FBB -42 149182F78003 -80 B8C75D08C365 -79 3412980827F4 -73 A47AA41D18B0 -76 CE2E48F50FA9 -58 0690A90EAA61 -29 BC2E48F50FA9 -60 6455B18A34D0 -66 1C497B133AAB -44 EAFCAF9C9905 -82 E8FCAF9C9904 -83 2C3033CDF2D2 -51 CE2E48F50FAA -82 BC2E48F50FAA -82 0290A90EAA63 -48 000D67369A83 -77 000D67369A82 -77 000D67369A81 -77 CAFEC0DEF00D -32


On Linux:

In [ ]:
WIFI_INTERFACE_NAME = 'wlan0'

def process_raw_lines_from_linux_iw(raw_lines):
    items = list()
    bss_pat = r"([a-z0-9]{2}:){5}[a-z0-9]{2}"
    rssi_pat = r"signal: (-[0-9]+.[0-9]+) dBm"
    
    start_indices = [i for i in range(len(raw_lines)) if re.search(bss_pat, raw_lines[i])]
    for i in range(len(start_indices)):
        start_index = start_indices[i]
        next_start_index = start_indices[i+1] if i+1 < len(start_indices) else len(raw_lines)
        bssid = re.search(bss_pat, raw_lines[start_index]).group(0).upper().replace(":", "")
        for j in range(start_index, next_start_index):
            m = re.search(rssi_pat, raw_lines[j])
            if m:
                rssi = m.group(1)
                break
        items.append(bssid + " " + rssi)
    return items


def get_bssids_and_rssis_via_linux_iw():
    iw_command = "sudo iw " + WIFI_INTERFACE_NAME + " scan"
    with os.popen(iw_command) as f:
        raw_lines = f.readlines()
    result_list = process_raw_lines_from_linux_iw(raw_lines)
    return " ".join(result_list)

bssids_and_rssis = get_bssids_and_rssis_via_linux_iw()

print bssids_and_rssis

The following command will run the localization program with the preceding bssids/rssis and print the output.  If any of your local bssids were found in the database, your coordinate estimate will be printed at the bottom.

In [11]:
localize_command = "./localization " + bssids_and_rssis
with os.popen(localize_command) as g:
    print g.read()

Querying database for the following BSSIDs:
BSSID 4C3275C4C4CC
RSSI -73
BSSID 2C3033C23FBB
RSSI -42
BSSID 149182F78003
RSSI -80
BSSID B8C75D08C365
RSSI -79
BSSID 3412980827F4
RSSI -73
BSSID A47AA41D18B0
RSSI -76
BSSID CE2E48F50FA9
RSSI -58
BSSID 0690A90EAA61
RSSI -29
BSSID BC2E48F50FA9
RSSI -60
BSSID 6455B18A34D0
RSSI -66
BSSID 1C497B133AAB
RSSI -44
BSSID EAFCAF9C9905
RSSI -82
BSSID E8FCAF9C9904
RSSI -83
BSSID 2C3033CDF2D2
RSSI -51
BSSID CE2E48F50FAA
RSSI -82
BSSID BC2E48F50FAA
RSSI -82
BSSID 0290A90EAA63
RSSI -48
BSSID 000D67369A83
RSSI -77
BSSID 000D67369A82
RSSI -77
BSSID 000D67369A81
RSSI -77
BSSID CAFEC0DEF00D
RSSI -32

Found record: 
_id = 295822003
bssid = CAFEC0DEF00D
manufactor = NULL
md5essid = NULL
latitude = 37.760849622041
longitude = -122.419518811837
source = openbmap
measurements = 98
last_updated = 2016-12-18
created_at = 2016-12-18

Found record: 
_id = 296505675
bssid = CAFEC0DEF00D
manufactor = NULL
md5essid = NULL
latitude = 37.760582671538
longitude = -122.4191723